In [43]:
%pip install --quiet --upgrade pip 
%pip install numpy --quiet
%pip install Pandas --quiet
%pip install sklearn --quiet
%pip install ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Horse Racing Results Predictor using Linear regression #

Predict `TodaysRaceCard.csv` using simple linear regression techniques by determining predicted **speed** of a horse in a given race based on its previous performance. 

Data used here is derived from the features extracted by the [Feature Analysis](https://github.com/LeeSanderson/RacingData/blob/main/Data/FeatureAnalysis.ipynb) notebook.

## Build the model ##

In [44]:
import numpy as np
import pandas as pd
import math
from abc import ABC, abstractmethod
from datetime import datetime, date

In [45]:
races = pd.read_csv("Race_Features.csv")
races['Off'] =  pd.to_datetime(races['Off'], format='%Y-%m-%d %H:%M:%S')
races.columns

Index(['RaceId', 'CourseId', 'RaceType', 'Off', 'DecimalOdds',
       'OfficialRating', 'RacingPostRating', 'TopSpeedRating',
       'DistanceInMeters', 'Going', 'Surface', 'HorseId', 'HorseName',
       'JockeyId', 'JockeyName', 'Age', 'HeadGear', 'RaceCardNumber',
       'StallNumber', 'WeightInPounds', 'FinishingPosition',
       'OverallBeatenDistance', 'RaceTimeInSeconds', 'Wins',
       'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 'Going_Firm',
       'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy',
       'Going_Soft', 'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other',
       'RaceType_SteepleChase', 'Speed', 'HorseCount', 'KnownHorseAndJockey',
       'NumberOfPriorRaces', 'LastRaceGoing', 'LastRaceSurface',
       'LastRaceDistanceInMeters', 'LastRaceWeightInPounds', 'LastRaceSpeed',
       'DaysRested', 'LastRaceDecimalOdds', 'LastRaceOfficialRating',
       'LastRaceRacingPostRating', 'LastRaceTopSpeedRating',
       'LastRaceAvgRelFinishi

In [46]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

race_info = ['RaceId', 'CourseId', 'Off', 'HorseId', 'HorseName', 'HorseCount', 'JockeyId', 'JockeyName', 'DecimalOdds', 'Wins', 'FinishingPosition']
predictors = ([
    'DistanceInMeters', 
    'WeightInPounds', 
    'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 
    'Going_Firm', 'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy', 'Going_Soft', 
    'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other', 'RaceType_SteepleChase', 

    'LastRaceDistanceInMeters', 
    'LastRaceWeightInPounds', 
    'LastRaceSpeed',
    'DaysRested',
    'LastRaceAvgRelFinishingPosition', 
    'LastRaceSurface_AllWeather', 'LastRaceSurface_Dirt', 'LastRaceSurface_Turf', 
    'LastRaceGoing_Good', 'LastRaceGoing_Good_To_Soft', 'LastRaceGoing_Soft', 'LastRaceGoing_Good_To_Firm', 'LastRaceGoing_Firm', 'LastRaceGoing_Heavy', 
    'LastRaceRaceType_Other', 'LastRaceRaceType_Hurdle', 'LastRaceRaceType_SteepleChase', 'LastRaceRaceType_Flat', 
    
    'JockeyNumberOfPriorRaces',
    'DaysSinceJockeyLastRaced',     
    'JockeyWinPercentage',
    'JockeyTop3Percentage',
    'JockeyAvgRelFinishingPosition'
    ])
prediction = ["Speed"]

train = races[race_info + predictors + prediction].dropna().copy()

In [47]:
# Cap rested days
train.loc[train["DaysRested"] > 10, "DaysRested"] = 10
train.loc[train["DaysSinceJockeyLastRaced"] > 10, "DaysSinceJockeyLastRaced"] = 10

# Cap prior races
# train.loc[train["JockeyNumberOfPriorRaces"] > 400] = 400 (negative effect)

In [48]:
# Now that we've dropped any rows with na values, calculate which races in which we can predict the speed for all horses.
train = train.drop("PredictableHorseCount", axis=1, errors='ignore')
groups = train.groupby(['RaceId']).apply(lambda g: pd.Series({'PredictableHorseCount': g['RaceId'].count()}, index=['PredictableHorseCount']))
train = pd.merge(train, groups, how='left', on=['RaceId'])

# 50% of races have 11 horse, 25% have 8 or less, 14% have 6 or less, 9% have 5 or less. Fewer horses should be more predictable
all_races_count = len(races["RaceId"].unique())
all_predictable = train[(train["HorseCount"] == train["PredictableHorseCount"]) & (train["HorseCount"] <= 5)]["RaceId"].unique().tolist()
all_predictable_count = len(all_predictable)
print(f"Possible predictable races = {all_predictable_count} out of {all_races_count} ({all_predictable_count * 100 / all_races_count}%)")

Possible predictable races = 3983 out of 37477 (10.627851749072764%)


In [49]:
from sklearn.model_selection import train_test_split

_, test_race_ids = train_test_split(all_predictable, test_size=0.2, random_state = 42)

test = train[train["RaceId"].isin(test_race_ids)]
train = train[train["RaceId"].isin(test_race_ids) == False]

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Create a pipeline that includes feature scaling and linear regression
model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2, interaction_only=False), LinearRegression())

In [51]:
inputs = train[predictors]
targets = train[prediction[0]].values

In [52]:
model.fit(inputs, targets)
test_inputs = test[predictors]
predictions = model.predict(test_inputs)

In [53]:
test_with_predictions = test.copy()
test_with_predictions["PredictedSpeed"] = predictions

In [54]:
test_with_predictions["PredictedRank"] = test_with_predictions.groupby("RaceId")["PredictedSpeed"].rank(method="dense", ascending=False)

In [55]:
correct_winners = test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] == 1)]

correct_winners_count = len(correct_winners)
incorrect_winners_count = len(test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] != 1)])

winnings = correct_winners["DecimalOdds"].sum()
win_percentage = 100.0 * correct_winners_count / incorrect_winners_count

print(f"Correct winners {correct_winners_count}, Incorrect winners {incorrect_winners_count}")
print(f"Win percentage {win_percentage}")
print(f"Winnings {winnings}, losses {incorrect_winners_count}, diff  {winnings - incorrect_winners_count}")

if win_percentage < 40:
    raise ValueError("Prediction rate too low")

Correct winners 258, Incorrect winners 540
Win percentage 47.77777777777778
Winnings 809.7587717837717, losses 540, diff  269.75877178377175


In [56]:
# Base = 23.3630494927759
# PolynomialFeatures(degree=2, interaction_only=True) = 25.01557632398754
# With PolynomialFeatures(degree=2, interaction_only=False) = 25.13252260679763
# With rest days capped =  25.210608424336975
# with max 11 horses in race = 28.612597776862906 (£467)
# with max 8 horses in race = 31.598513011152416 (£336)
# with max 6 horses in race = 38.113207547169814 (£343)
# with max 5 horses in race = 47.77777777777778 (£269)

## Predictions ##

In [57]:
todays_races = pd.read_csv("TodaysRaceCards.csv")
todays_races['Off'] =  pd.to_datetime(todays_races['Off'], format='%m/%d/%Y %H:%M:%S')
todays_races.columns

Index(['RaceId', 'RaceName', 'CourseId', 'CourseName', 'Off', 'RaceType',
       'Class', 'Pattern', 'RatingBand', 'AgeBand', 'SexRestriction',
       'Distance', 'DistanceInFurlongs', 'DistanceInMeters', 'DistanceInYards',
       'Going', 'Surface', 'HorseId', 'HorseName', 'JockeyId', 'JockeyName',
       'TrainerId', 'TrainerName', 'Age', 'HeadGear', 'RaceCardNumber',
       'StallNumber', 'Weight', 'WeightInPounds', 'FractionalOdds',
       'DecimalOdds', 'OfficialRating', 'RacingPostRating', 'TopSpeedRating'],
      dtype='object')

### Expand categorical variables

In [58]:
surface_categories = ["Surface_AllWeather", "Surface_Dirt", "Surface_Turf"]
todays_races = todays_races.drop(surface_categories + ["Surface_Unknown"], axis=1, errors='ignore')
todays_races["SurfaceTemp"] = todays_races["Surface"]
todays_races = pd.get_dummies(todays_races, prefix="Surface", columns=["SurfaceTemp"], dtype=float)
todays_races = todays_races.drop("Surface_Unknown", axis=1, errors='ignore') # Drop unknown surface as only small number.
for surface in surface_categories:
    todays_races[surface] = todays_races.get(surface, 0.0)
todays_races[surface_categories].value_counts()

Surface_AllWeather  Surface_Dirt  Surface_Turf
0.0                 0.0           1.0             396
1.0                 0.0           0.0              83
dtype: int64

In [59]:
# Normalise going based on rules here: https://www.racingpost.com/guide-to-racing/what-is-the-going-ann7h6W6VB3b/
# Values should be: Firm, Good_To_Firm, Good, Good_To_Soft, Soft, Heavy
norm_map = ({
    "Good": "Good", 
    "Standard": "Good",
    "Soft": "Soft",
    "Good To Soft": "Good_To_Soft",
    "Good To Firm": "Good_To_Firm",
    "Heavy": "Heavy",
    "Good To Yielding": "Good_To_Soft",    
    "Yielding": "Good_To_Soft",
    "Standard To Slow": "Good_To_Soft",  
    "Very Soft": "Heavy",
    "Fast": "Firm",
    "Firm": "Firm",
    "Soft To Heavy": "Heavy",    
    "Yielding To Soft": "Soft",
    "Slow": "Soft",
    "Sloppy": "Heavy",
    "Muddy": "Heavy",
    "Frozen": "Heavy"
})

todays_races["NormGoing"] = todays_races["Going"].map(norm_map)

In [60]:
going_categories  = ["Going_Good", "Going_Good_To_Soft", "Going_Soft", "Going_Good_To_Firm", "Going_Firm", "Going_Heavy"]
todays_races = todays_races.drop(going_categories, axis=1, errors='ignore')
todays_races = pd.get_dummies(todays_races, prefix="Going", columns=["NormGoing"], dtype=float)
for going in going_categories:
    todays_races[going] = todays_races.get(going, 0.0)

todays_races[going_categories].value_counts()

Going_Good  Going_Good_To_Soft  Going_Soft  Going_Good_To_Firm  Going_Firm  Going_Heavy
1.0         0.0                 0.0         0.0                 0.0         0.0            201
0.0         0.0                 1.0         0.0                 0.0         0.0            177
            1.0                 0.0         0.0                 0.0         0.0            101
dtype: int64

In [61]:
race_type_categories = ["RaceType_Other", "RaceType_Hurdle", "RaceType_SteepleChase", "RaceType_Flat"]
todays_races = todays_races.drop(race_type_categories, axis=1, errors='ignore')
todays_races["RaceTypeTemp"] = todays_races["RaceType"]
todays_races = pd.get_dummies(todays_races, prefix="RaceType", columns=["RaceTypeTemp"], dtype=float)
for race_type in race_type_categories:
    todays_races[race_type] = todays_races.get(race_type, 0.0)

todays_races[race_type_categories].value_counts()

RaceType_Other  RaceType_Hurdle  RaceType_SteepleChase  RaceType_Flat
1.0             0.0              0.0                    0.0              317
0.0             1.0              0.0                    0.0              135
                0.0              0.0                    1.0               16
                                 1.0                    0.0               11
dtype: int64

### Load horse stats

In [62]:
horse_stats = pd.read_csv("Horse_Stats.csv")
horse_stats['LastOff'] =  pd.to_datetime(horse_stats['LastOff'], format='%Y-%m-%d %H:%M:%S')
horse_stats.columns

Index(['HorseId', 'LastOff', 'LastRaceDistanceInMeters',
       'LastRaceWeightInPounds', 'LastRaceAvgRelFinishingPosition',
       'LastRaceSurface_AllWeather', 'LastRaceSurface_Dirt',
       'LastRaceSurface_Turf', 'LastRaceGoing_Firm', 'LastRaceGoing_Good',
       'LastRaceGoing_Good_To_Firm', 'LastRaceGoing_Good_To_Soft',
       'LastRaceGoing_Heavy', 'LastRaceGoing_Soft', 'LastRaceRaceType_Flat',
       'LastRaceRaceType_Hurdle', 'LastRaceRaceType_Other',
       'LastRaceRaceType_SteepleChase', 'LastRaceSpeed'],
      dtype='object')

In [63]:
todays_races = pd.merge(todays_races, horse_stats, how="left", on=["HorseId"])

In [64]:
today = np.datetime64(datetime.today())
one_day = np.timedelta64(1, 'D')
todays_races["DaysRested"] = np.ceil((today - todays_races["LastOff"]) / one_day)
todays_races.loc[todays_races["DaysRested"] > 10, "DaysRested"] = 10
todays_races = todays_races.drop("LastOff", axis=1, errors='ignore')

In [65]:
groups = todays_races.groupby(['RaceId']).apply(lambda g: pd.Series({'HorseCount': g['RaceId'].count()}, index=['HorseCount']))
todays_races = pd.merge(todays_races, groups, how='left', on=['RaceId'])

### Load jockey stats

In [66]:
jockey_stats = pd.read_csv("Jockey_Stats.csv")
jockey_stats['LastOff'] =  pd.to_datetime(jockey_stats['LastOff'], format='%Y-%m-%d %H:%M:%S')
jockey_stats.columns

Index(['JockeyId', 'LastOff', 'JockeyNumberOfPriorRaces',
       'JockeyWinPercentage', 'JockeyTop3Percentage',
       'JockeyAvgRelFinishingPosition'],
      dtype='object')

In [67]:
todays_races = pd.merge(todays_races, jockey_stats, how="left", on=["JockeyId"])

In [68]:
todays_races["DaysSinceJockeyLastRaced"] = np.ceil((today - todays_races["LastOff"]) / one_day)
todays_races.loc[todays_races["DaysSinceJockeyLastRaced"] > 10, "DaysSinceJockeyLastRaced"] = 10
todays_races = todays_races.drop("LastOff", axis=1, errors='ignore')

### Eliminate races that cannot be reliably predicted

In [69]:
race_info = ['RaceId', 'CourseId', 'CourseName', 'Off', 'HorseId', 'HorseName', 'HorseCount', 'JockeyId', 'JockeyName']

races_to_predict = todays_races[race_info + predictors].dropna().copy()

if len(races_to_predict) > 0:
    # Now that we've dropped any rows with na values, calculate which races in which we can predict the speed for all horses.
    races_to_predict = races_to_predict.drop("PredictableHorseCount", axis=1, errors='ignore')
    groups = races_to_predict.groupby(['RaceId']).apply(lambda g: pd.Series({'PredictableHorseCount': g['RaceId'].count()}, index=['PredictableHorseCount']))
    races_to_predict = pd.merge(races_to_predict, groups, how='left', on=['RaceId'])
    races_to_predict = races_to_predict[(races_to_predict["HorseCount"] == races_to_predict["PredictableHorseCount"]) & (races_to_predict["HorseCount"] <= 6)]


,RaceId,CourseId,CourseName,Off,HorseId,HorseName,HorseCount,JockeyId,JockeyName,DistanceInMeters,WeightInPounds,Surface_AllWeather,Surface_Dirt,Surface_Turf,Going_Firm,Going_Good,Going_Good_To_Firm,Going_Good_To_Soft,Going_Heavy,Going_Soft,RaceType_Flat,RaceType_Hurdle,RaceType_Other,RaceType_SteepleChase,LastRaceDistanceInMeters,LastRaceWeightInPounds,LastRaceSpeed,DaysRested,LastRaceAvgRelFinishingPosition,LastRaceSurface_AllWeather,LastRaceSurface_Dirt,LastRaceSurface_Turf,LastRaceGoing_Good,LastRaceGoing_Good_To_Soft,LastRaceGoing_Soft,LastRaceGoing_Good_To_Firm,LastRaceGoing_Firm,LastRaceGoing_Heavy,LastRaceRaceType_Other,LastRaceRaceType_Hurdle,LastRaceRaceType_SteepleChase,LastRaceRaceType_Flat,JockeyNumberOfPriorRaces,DaysSinceJockeyLastRaced,JockeyWinPercentage,JockeyTop3Percentage,JockeyAvgRelFinishingPosition,PredictableHorseCount
95,847720,7,Brighton,2023-09-18 15:40:00,1495066,Still Standing,6,101759,Taylor Fisher,2412,160,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2010.0,163.0,14.685468,10.0,0.708585,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,419.0,9.0,0.128878,0.369928,0.526857,6
96,847720,7,Brighton,2023-09-18 15:40:00,2455834,Sword Beach,6,86013,David Probert,2412,156,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2412.0,161.0,16.124072,10.0,0.561185,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2414.0,9.0,0.130075,0.365783,0.540205,6
97,847720,7,Brighton,2023-09-18 15:40:00,4628835,Plus Point,6,94575,George Wood,2412,153,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2010.0,150.0,15.518839,10.0,0.554701,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,389.0,10.0,0.084833,0.293059,0.591069,6
98,847720,7,Brighton,2023-09-18 15:40:00,4484933,Robusto,6,84857,Luke Morris,2412,152,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2412.0,151.0,16.205321,10.0,0.530291,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2408.0,9.0,0.087625,0.315615,0.578233,6
99,847720,7,Brighton,2023-09-18 15:40:00,3382815,Cloudy Rose,6,96974,Adam Farragher,2412,140,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2412.0,148.0,15.652174,10.0,0.511441,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,481.0,8.0,0.133056,0.349272,0.536835,6
100,847720,7,Brighton,2023-09-18 15:40:00,3096282,On The Nose,6,100531,Aidan Keeley,2412,139,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2010.0,145.0,15.663965,10.0,0.624609,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,564.0,8.0,0.092199,0.312057,0.584342,6
127,847880,101,Worcester,2023-09-18 14:05:00,2875013,Grand Roi,4,93313,Mr Alex Chadwick,3317,186,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3820.0,188.0,13.337989,10.0,0.608862,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,38.0,10.0,0.184211,0.421053,0.599078,4
128,847880,101,Worcester,2023-09-18 14:05:00,3432677,Fringill Dike,4,93541,Charlie Hammond,3317,176,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3217.0,192.0,14.105319,10.0,0.396146,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,493.0,8.0,0.158215,0.476673,0.573409,4
129,847880,101,Worcester,2023-09-18 14:05:00,3030298,Glory And Honour,4,87958,Jonathan England,3317,167,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3418.0,172.0,14.128054,9.0,0.562135,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,255.0,9.0,0.168627,0.498039,0.563072,4
130,847880,101,Worcester,2023-09-18 14:05:00,1266706,With A Start,4,91230,James Best,3317,165,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3418.0,166.0,12.159422,10.0,0.789116,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,572.0,10.0,0.066434,0.309441,0.679678,4


In [70]:
test_inputs = races_to_predict[predictors]
predictions = model.predict(test_inputs)
races_to_predict["PredictedSpeed"] = predictions

In [71]:
races_to_predict["PredictedRank"] = races_to_predict.groupby("RaceId")["PredictedSpeed"].rank(method="dense", ascending=False)

In [74]:
races_to_predict[races_to_predict["PredictedRank"] == 1].sort_values(["CourseName", "Off"])

,RaceId,CourseId,CourseName,Off,HorseId,HorseName,HorseCount,JockeyId,JockeyName,DistanceInMeters,WeightInPounds,Surface_AllWeather,Surface_Dirt,Surface_Turf,Going_Firm,Going_Good,Going_Good_To_Firm,Going_Good_To_Soft,Going_Heavy,Going_Soft,RaceType_Flat,RaceType_Hurdle,RaceType_Other,RaceType_SteepleChase,LastRaceDistanceInMeters,LastRaceWeightInPounds,LastRaceSpeed,DaysRested,LastRaceAvgRelFinishingPosition,LastRaceSurface_AllWeather,LastRaceSurface_Dirt,LastRaceSurface_Turf,LastRaceGoing_Good,LastRaceGoing_Good_To_Soft,LastRaceGoing_Soft,LastRaceGoing_Good_To_Firm,LastRaceGoing_Firm,LastRaceGoing_Heavy,LastRaceRaceType_Other,LastRaceRaceType_Hurdle,LastRaceRaceType_SteepleChase,LastRaceRaceType_Flat,JockeyNumberOfPriorRaces,DaysSinceJockeyLastRaced,JockeyWinPercentage,JockeyTop3Percentage,JockeyAvgRelFinishingPosition,PredictableHorseCount,PredictedSpeed,PredictedRank
99,847720,7,Brighton,2023-09-18 15:40:00,3382815,Cloudy Rose,6,96974,Adam Farragher,2412,140,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2412.0,148.0,15.652174,10.0,0.511441,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,481.0,8.0,0.133056,0.349272,0.536835,6,15.708817,1.0
317,849918,182,Fairyhouse,2023-09-18 17:55:00,5009826,Velvet Skies,6,100805,Wesley Joyce,1206,140,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1407.0,138.0,15.775311,10.0,0.650579,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,322.0,8.0,0.062112,0.245342,0.538423,6,15.592606,1.0
129,847880,101,Worcester,2023-09-18 14:05:00,3030298,Glory And Honour,4,87958,Jonathan England,3317,167,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3418.0,172.0,14.128054,9.0,0.562135,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,255.0,9.0,0.168627,0.498039,0.563072,4,13.791458,1.0
